In [1]:
%load_ext autoreload
%autoreload 2

## Feature extraction

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os 

In [3]:
import sys

def get_n_dir_up(path, n):
    for _ in range(n):
        path = os.path.dirname(path)
    return path

CUR_PATH= os.path.abspath("__file__")
sys.path.append(os.path.join(get_n_dir_up(CUR_PATH, 2)))

In [4]:
#experiment specific params

EVENTS_ARR = [
    ('fixation', 1000),
    ('stimulus1', 250),	
    ('cue1', 500),	
    ('mask1', 500),
    ('shortDelay', 1000),
    ('stimulus2', 250),
    ('cue2', 500),
    ('mask2', 500),
    ('longDelay', 5000),
]

included = np.sum([t[1] for t in EVENTS_ARR])

In [5]:
from utils.eye_trial import generate_events
from utils.eye_trial import generate_stim_phases

EVENTS = generate_events()
ALL_STIM_PHASES = generate_stim_phases(EVENTS)

## Read data

In [6]:
def read_cleaned_data(behavior_folder, gaze_folder, filename):
    psyFull_path = f'{behavior_folder}/{filename}.csv'
    psyFull = None
    if os.path.exists(psyFull_path):
        psyFull = pd.read_csv(psyFull_path)
        psyFull = psyFull.drop(
            columns=[col for col in psyFull.columns if col.startswith("Unnamed")])
        
    gazeClean_path = f'{gaze_folder}/{filename}.csv'
    gazeClean = None
    if os.path.exists(gazeClean_path):
        gazeClean = pd.read_csv(gazeClean_path)
        gazeClean = gazeClean.drop(
            columns=[col for col in gazeClean.columns if col.startswith("Unnamed")])

    return psyFull, gazeClean

### Define Feature Extractor

- mean gaze position
- saccade direction
- 1d angles of gaze position
- 2d heat maps of gaze position

In [7]:
from abc import ABC, abstractmethod
from utils.eye_data import XYData

class BaseEyeFeatureExtractor(ABC):
    def __init__(self, settings):
        self.settings = settings
        self.initialize()

    def center_normalize(self, all_xs, all_ys, norm_xs, norm_ys, center_align_setting):
        center = center_align_setting['center']
        norm_center = center
        if center_align_setting.get('median_align', False):
            # instead we use subject median position
            timepoint_align = center_align_setting.get(
                'timepoint_align', False)
            if timepoint_align:
                center = (
                    np.median(all_xs, axis=0).astype(int),
                    np.median(all_ys, axis=0).astype(int)
                )
                norm_center = (
                    np.median(norm_xs, axis=0).astype(int),
                    np.median(norm_ys, axis=0).astype(int)
                )
            else:
                center = (int(np.median(norm_xs)), int(np.median(norm_ys)))
                norm_center = center
        all_xs, all_ys = all_xs - center[0], all_ys - center[1]
        norm_xs, norm_ys = norm_xs - norm_center[0], norm_ys - norm_center[1]

        return all_xs, all_ys, norm_xs, norm_ys

    @abstractmethod
    def initialize(self):
        pass

    @abstractmethod
    def get_subject_features(self, source_data: XYData, **kwargs):
        pass

#### mean gaze position

In [8]:
class GazeMeanExtractor(BaseEyeFeatureExtractor):
    def initialize(self):
        pass

    def bin_data(self, data):
        time_bin = self.settings['time_binning']['timebin_size']
        # assuming the last dimension is time dimension
        shape_original = data.shape
        t_original = shape_original[-1]
        n_bins = t_original // time_bin
        data = data[..., :n_bins * time_bin]
        data = data.reshape(*shape_original[:-1], n_bins, time_bin)
        return data

    def convert_to_avg_position(self, xs, ys):
        # binning
        xs = self.bin_data(xs)
        ys = self.bin_data(ys)

        # compute mean position
        xs = np.mean(xs, axis=-1)
        ys = np.mean(ys, axis=-1)

        results = np.stack([xs, ys], axis=-1)

        return results


    def get_subject_features(self, source_data: XYData, norm_phases, trial_ids):
        # apply centering
        all_xs, all_ys = source_data.read(trial_ids=trial_ids)
        norm_xs, norm_ys = source_data.read_phase(phases=norm_phases, trial_ids=trial_ids)
        center_align_setting = self.settings['center_align']

        # converted position to z score
        all_xs, all_ys, norm_xs, norm_ys = self.center_normalize(
            all_xs, all_ys, norm_xs, norm_ys, center_align_setting)
        
        if center_align_setting['z_score']:            
            # first apply boundary
            radius = center_align_setting['z_score']['radius'] 
            norm_xs, norm_ys = np.clip(norm_xs, -radius, radius), np.clip(norm_ys, -radius, radius)
            
            # then normalization
            mean_norm_xs, mean_norm_ys = np.mean(norm_xs), np.mean(norm_ys)
            std_norm_xs, std_norm_ys = np.std(norm_xs), np.std(norm_ys)
            norm_xs, norm_ys = (norm_xs - mean_norm_xs) / std_norm_xs, (norm_ys - mean_norm_ys) / std_norm_ys
            all_xs, all_ys = (all_xs - mean_norm_xs) / std_norm_xs, (all_ys - mean_norm_ys) / std_norm_ys

            # finally clip to remove too large values
            cap_ratio = center_align_setting['z_score']['cap_ratio']
            all_xs, all_ys = np.clip(all_xs, -cap_ratio, cap_ratio), np.clip(all_ys, -cap_ratio, cap_ratio)

        # compute mean position
        results = self.convert_to_avg_position(all_xs, all_ys)
        
        return results


#### 1d feature

In [9]:
from utils.eye_feature import SaccadeAngleStats
from scipy.ndimage import gaussian_filter1d

class GazeAngleFeatureExtractor(BaseEyeFeatureExtractor):
    def initialize(self):
        self.GazeAngleHelper = SaccadeAngleStats(self.settings)

    def bin_data(self, data):
        time_bin = self.settings['time_binning']['timebin_size']
        # assuming the last dimension is time dimension
        shape_original = data.shape
        t_original = shape_original[-1]
        n_bins = t_original // time_bin
        data = data[..., :n_bins * time_bin]
        data = data.reshape(*shape_original[:-1], n_bins, time_bin)
        return data

    def angle_to_ids(self, angles):
        n_angle_bins = self.settings['angle_to_id']['n_angle_bins']
        to_fold = self.settings['angle_to_id']['to_fold']
        epoch = 180 if to_fold else 360
        angles = angles % epoch
        angle_bin_size = epoch / n_angle_bins 
        angle_bin_ids = (angles + angle_bin_size/2) / angle_bin_size
        angle_bin_ids = angle_bin_ids.astype(int) % n_angle_bins
        # print(angles[:2, :2, :3], angle_bin_ids[:2, :2, :3])
        return angle_bin_ids
    
    def batch_process_1dids(self, state_ids, weights):
        filter_zero = self.settings['occurence']['filter_zero']
        n_states = self.settings['angle_to_id']['n_angle_bins']
        
        assert len(state_ids.shape) == 3 
        n_trials, n_time_bins, timebin_size = state_ids.shape
        
        # compute number of occurence each trial, each time point
        occurences = None
        if filter_zero:
            occurences = np.sum(weights>0, axis=-1, keepdims=True)
        else:
            occurences = np.ones((n_trials, n_time_bins, 1)) * timebin_size

        # aggregate
        # output: n_trials * n_time_bins * n_states
        results = np.zeros((n_trials, n_time_bins, n_states))
        trial_indices, time_indices, state_indices = np.meshgrid(
            np.arange(n_trials), np.arange(n_time_bins), np.arange(timebin_size), indexing='ij'
        )
        np.add.at(
            results, 
            (trial_indices.ravel(), time_indices.ravel(), state_ids.ravel()), 
            weights.ravel())

        # normalize
        occ_non_zero = np.where(occurences>0, occurences, 1)
        results = results / occ_non_zero

        return results
    
    def convert_collapsed_1dvec(self, xs, ys):
        _, angles, mag_weights = self.GazeAngleHelper.convert_subject_occurence_to_angle_weight(xs, ys)
        # sum everything up
        angle_ids = self.angle_to_ids(angles)
        n_states = self.settings['angle_to_id']['n_angle_bins']
        h = np.zeros(n_states)
        np.add.at(h, angle_ids.flatten(), mag_weights.flatten())
        # normalization: count number of occurences
        filter_zero = self.settings['occurence']['filter_zero']
        n_occurences = np.sum(mag_weights > 0).astype(int) if filter_zero else np.size(xs)
        h = h / n_occurences

        return h
    
    def convert_eyedata_1dvec(self, xs, ys):
        # convert all angle, mag for each timepoint
        _, angles, mag_weights = self.GazeAngleHelper.convert_subject_occurence_to_angle_weight(xs, ys)
        # print('convert to angle', xs[:2, :2], ys[:2, :2], angles[:2, :2])
    
        # aggreagte each time bin
        angles_binned = self.bin_data(angles)
        mag_weights_binned = self.bin_data(mag_weights)

        # convert angle to angle bin id
        angle_ids_binned = self.angle_to_ids(angles_binned)

        # now compute the collapsed
        results = self.batch_process_1dids(angle_ids_binned, mag_weights_binned)

        return results
        
    def get_subject_features(self, source_data: XYData, norm_phases, trial_ids):
        # apply center normalization if needed
        all_xs, all_ys = source_data.read(trial_ids=trial_ids)
        norm_xs, norm_ys = source_data.read_phase(phases=norm_phases, trial_ids=trial_ids)
        center_align_setting = self.settings['center_align']

        # first move everything to the center
        all_xs, all_ys, norm_xs, norm_ys = self.center_normalize(
            all_xs, all_ys, norm_xs, norm_ys, center_align_setting)            
        
        # NOTE: input data should have been subject-normalized
        results = self.convert_eyedata_1dvec(all_xs, all_ys)

        # also compute the subject bias, using the norm_phases
        subj_vec = self.convert_collapsed_1dvec(norm_xs, norm_ys)
        
        # normalization: remove subject bias
        results = results - subj_vec

        # finally, apply smoothing
        smoothing_params = self.settings.get('smoothing')
        if smoothing_params is not None:
            results = gaussian_filter1d(
                results, 
                sigma=smoothing_params['sigma'], 
                mode='wrap', axis=-1)
        
        return results

#### 2d feature

In [10]:
from utils.eye_stats import compute_vecmap
from utils.eye_preprocess import convert_movement_to_angle
from scipy.ndimage import gaussian_filter

class Map2dHelper:
    def __init__(self, settings):
        # process settings
        self.min_mag = settings['min_mag']
        self.max_mag = settings['max_mag']
        self.smoothing_params = settings['smoothing']
        self.vec_map_params = settings['vecmap']

    def filter_by_mags(self, xs, ys):
        # filter out too tiny or too large magnitude
        _, mags = convert_movement_to_angle(xs, ys, compute_mag=True)
        mag_mask = (mags >= self.min_mag) & (mags <= self.max_mag)
        xs = xs[mag_mask]
        ys = ys[mag_mask]
        return xs, ys

    def smooth_vecmap(self, vecmap):
        sigma = self.smoothing_params['sigma']
        vecmap = gaussian_filter(vecmap, sigma=sigma, mode='constant', cval=0)
        return vecmap

    def data_to_vecmaps(self, xs, ys):
        vecmap_settings = self.vec_map_params
        h, _, _ = compute_vecmap(xs, ys, **vecmap_settings)
        return h
    
    def convert_xy_to_feature(self, xs, ys):
        # filteirng
        xs, ys = self.filter_by_mags(xs, ys)
        # 2d map
        vecmap = self.data_to_vecmaps(xs, ys)
        # smoothing
        vecmap = self.smooth_vecmap(vecmap)
        # flattening
        vec = vecmap.flatten()
        # normalization
        vec_sum = np.sum(vec)
        vec_sum = vec_sum if vec_sum > 0 else 1
        vec = vec / vec_sum
        return vec

class GazeHeatmapFeatureExtractor(BaseEyeFeatureExtractor):
    def initialize(self):
        self.vecmap_helper = Map2dHelper(self.settings['2dhist'])

    def bin_data(self, data):
        time_bin = self.settings['time_binning']['timebin_size']
        # assuming the last dimension is time dimension
        shape_original = data.shape
        t_original = shape_original[-1]
        n_bins = t_original // time_bin
        data = data[..., :n_bins * time_bin]
        data = data.reshape(*shape_original[:-1], n_bins, time_bin)
        return data

    def convert_collapsed_vec(self, xs, ys):
        xs = xs.flatten()
        ys = ys.flatten()
        vec = self.vecmap_helper.convert_xy_to_feature(xs, ys)
        return vec

    def convert_bulk_vec(self, xs, ys):
        # aggreagte each time bin
        xs = self.bin_data(xs)
        ys = self.bin_data(ys)
        n_trials, n_time_bins, timebin_size = xs.shape
        # aggregate features
        results = []
        for trial_id in range(n_trials):
            trial_result = []
            for tid in range(n_time_bins):
                tresult = self.convert_collapsed_vec(
                    xs[trial_id][tid],
                    ys[trial_id][tid])
                trial_result.append(tresult)
            results.append(trial_result)
        results = np.array(results)
        return results
        
    def get_subject_features(self, source_data: XYData, norm_phases, trial_ids):
        # apply center normalization if needed
        all_xs, all_ys = source_data.read(trial_ids=trial_ids)
        norm_xs, norm_ys = source_data.read_phase(phases=norm_phases, trial_ids=trial_ids)
        center_align_setting = self.settings['center_align']

        # first move everything to the center
        all_xs, all_ys, norm_xs, norm_ys = self.center_normalize(
            all_xs, all_ys, norm_xs, norm_ys, center_align_setting)           
        
        # NOTE: input data should have been subject-normalized
        results = self.convert_bulk_vec(all_xs, all_ys)

        # also compute the subject bias, using the norm_phases
        subj_vec = self.convert_collapsed_vec(norm_xs, norm_ys)
        
        # normalization: remove subject bias
        results = results - subj_vec

        return results

#### check feature extraction

**comment**: to get the version not align at each timepont, just comment out 'timepoint_align' (but for feature extraction we need to set it true to address the shifting)

In [11]:
# mean position
DEFAULT_MEAN_POS_SETTINGS = {
    'center_align' : {
        'center': (960, 540),
        'median_align': True,
        'timepoint_align': True,
        'z_score': {
            'radius': 200,
            'cap_ratio': 4,
        },
    },
    'time_binning': {
        'timebin_size': 50,
    }
}

In [12]:
# 1d
DEFAULT_1D_VEC_SETTINGS = {
    'occurence': {
        'n_angle_bins':360,
        'n_mag_bins': 10,
        'min_mag_thresh': 15,
        'max_mag_thresh': 150,
        'log_transform': False,
        'filter_zero': False,
    }, # how to filter event data
    'angle_to_id': {
        'n_angle_bins': 30,
        'to_fold': False,
    }, # how to convert angle to state id
    'time_binning': {
        'timebin_size': 50,
    },
    'dist_metric': 'cos',
    'center_align' : {
        'center': (960, 540),
        'median_align': True,
        'timepoint_align': True,
    },
    'smoothing': {
        'sigma': 2,
    }
}



In [13]:
DEFAULT_2D_VEC_SETTINGS = {
    '2dhist': { 
        'min_mag': 15,
        'max_mag': 300,
        'vecmap': {
            'x_center': 0,
            'y_center': 0,
            'x_radius': 150, # 80,
            'y_radius': 150, # 80,
            'n_bins': 15, # 20,
            'log_transform': False,
        },
        'smoothing': {
            'sigma': 3,
        },
    },
    'time_binning': {
        'timebin_size': 50,
    },
    'dist_metric': 'cos', # 'euc',
    'center_align' : {
        'center': (960, 540),
        'median_align': True,
        'timepoint_align': True,
    },
}

In [14]:
feature_extractor = GazeMeanExtractor(DEFAULT_MEAN_POS_SETTINGS)
# feature_extractor = GazeAngleFeatureExtractor(DEFAULT_1D_VEC_SETTINGS)
# feature_extractor = GazeHeatmapFeatureExtractor(DEFAULT_2D_VEC_SETTINGS)

In [15]:
NORM_PHASES = [
    ALL_STIM_PHASES[0]['display'],
    ALL_STIM_PHASES[0]['delay'],
    ALL_STIM_PHASES[1]['display'],
    ALL_STIM_PHASES[1]['delay'],
] # define the phasese we use for normalization

In [16]:
def read_one_subject_feature(fe: BaseEyeFeatureExtractor, eyedata, df, subj):
    subj_df = df[df['participant'] == subj]
    subj_trial_ids_temp = subj_df['TRIALID'].to_numpy()

    # read subject trial gaze data
    subj_xs, subj_ys, subj_trial_mask = eyedata.read(
        trial_ids=subj_trial_ids_temp, get_trial_mask=True)
    subj_trial_ids = subj_trial_ids_temp[subj_trial_mask]
    subj_eyedata = XYData(subj_xs, subj_ys, subj_trial_ids)

    # collect feature data
    subj_features = fe.get_subject_features(
        source_data=subj_eyedata, norm_phases=NORM_PHASES, trial_ids=None)

    # read valid ids
    subj_df = subj_df[subj_trial_mask]
    subj_trial_ids = subj_trial_ids_temp[subj_trial_mask]

    return subj_features, subj_trial_mask, subj_df, subj_trial_ids

In [17]:
DATA_FOLDER = os.path.join(get_n_dir_up(CUR_PATH, 3), 'data')
DATA_LARGE_FOLDER = os.path.join(get_n_dir_up(CUR_PATH, 4), 'data', 'pilot')
DEFAULT_BEHAV_FOLDER = os.path.join(DATA_FOLDER, 'behavior', 'batches')
DEFAULT_GAZE_FOLDER = os.path.join(DATA_LARGE_FOLDER, 'gaze')

In [18]:
import json

with open(os.path.join(DATA_FOLDER, 'batch.json')) as f:
    SUBJ_MAPPING = json.load(f)
    SUBJ_DATA_IDS = list(SUBJ_MAPPING.keys())
    SUBJ_DATA_IDS.sort()

SUBJ_DATA_IDS

['908to915', '916to922', '927to937', '938to949', '950to952']

In [19]:
AllPsyFull, AllGazeCleans = [], []
AllSubjBatches = []

for subj_data_id in SUBJ_DATA_IDS:
    psyFull, gazeClean = read_cleaned_data(
        DEFAULT_BEHAV_FOLDER, DEFAULT_GAZE_FOLDER, subj_data_id)
    
    if (psyFull is not None) and (gazeClean is not None):
        AllPsyFull.append(psyFull)
        AllGazeCleans.append(gazeClean)
        AllSubjBatches.append(SUBJ_MAPPING[subj_data_id])

#### pad the PsyFull data

In [20]:
def get_last_resp_stim(row):
    if pd.isna(row['resp_1_last_drawing_tend']) and pd.isna(row['resp_2_last_drawing_tend']):
        return np.nan
    elif pd.isna(row['resp_1_last_drawing_tend']):
        return row['stim_2']
    elif pd.isna(row['resp_2_last_drawing_tend']):
        return row['stim_1']
    else:
        return row['stim_2'] if row['resp_2_last_drawing_tend'] > row['resp_1_last_drawing_tend'] else row['stim_1']

def get_last_response(row):
    if pd.isna(row['resp_1_last_drawing_tend']) and pd.isna(row['resp_2_last_drawing_tend']):
        return np.nan
    elif pd.isna(row['resp_1_last_drawing_tend']):
        return row['resp_2']
    elif pd.isna(row['resp_2_last_drawing_tend']):
        return row['resp_1']
    else:
        return row['resp_2'] if row['resp_2_last_drawing_tend'] > row['resp_1_last_drawing_tend'] else row['resp_1']

for psyFull in AllPsyFull:
    # last resp
    psyFull['current_last_response'] = psyFull.apply(get_last_response, axis=1)
    # stim of last resp
    psyFull['current_last_resp_stim'] = psyFull.apply(get_last_resp_stim, axis=1)

In [21]:
def copy_from_last_trial(df, src_name, new_name):
    df_copy = df.copy()
    df_copy['trial'] += 1  # Increment trial to match the next row's trial
    df_copy = df_copy.rename(columns={src_name: new_name})

    df_copy = df[['participant', 'block', 'trial', src_name]].copy()
    df_copy['trial'] += 1  # Increment trial to match the next row's trial
    df_copy = df_copy.rename(columns={src_name: new_name})
    df = df.merge(df_copy, on=['participant', 'block', 'trial'], how='left')
    return df

AllPsyFullCopy = []
for psyFull in AllPsyFull:
    # copy from last trial resp
    psyFull = copy_from_last_trial(
        psyFull, 'current_last_response', 'prev_last_response')
    # copy from last trial stim of last resp
    psyFull = copy_from_last_trial(
        psyFull, 'current_last_resp_stim', 'prev_last_resp_stim')
    # copy from last trial stims
    psyFull = copy_from_last_trial(
        psyFull, 'stim_1', 'prev_stim_1')
    psyFull = copy_from_last_trial(
        psyFull, 'stim_2', 'prev_stim_2')
    
    # merge it
    AllPsyFullCopy.append(psyFull)
AllPsyFull = AllPsyFullCopy

In [22]:
example_behav_df = AllPsyFull[0]
example_gaze_df = AllGazeCleans[0]  

In [23]:
from utils.eye_data import create_xydata_from_df

example_eye_data = create_xydata_from_df(example_gaze_df)
subj908_features, subj908_mask, subj908_df, subj908_trial_ids  = read_one_subject_feature(
    feature_extractor, example_eye_data, example_behav_df, 908
)

In [24]:
subj908_features.shape

(160, 200, 2)

### Save features extracted

- for each subject
    - for each feature type: generate a feature map

- combination:
    - subject - time point - feature combined
    - a dictionary for mapping of feature id to feature and settings

In [25]:
DEFAULT_FEATURE_FOLDER = os.path.join(get_n_dir_up(CUR_PATH, 1), 'features')

In [26]:
import hashlib

def hash_configs(config, n_digits=6):
    config_str = json.dumps(config, sort_keys=True)  # Ensure consistent order
    config_hash = hashlib.sha256(config_str.encode('utf-8')).hexdigest()
    config_hash = config_hash[:n_digits]
    return config_hash

In [27]:
def generate_combined_features_for_one_subjects(
        feature_extractor_classes, feature_extractor_settings,
        source_eyedata, source_behav_df, subj_id, overwrite=False):
    
    subj_result_folder = os.path.join(DEFAULT_FEATURE_FOLDER, f'{subj_id}')
    if not os.path.exists(subj_result_folder):
        os.makedirs(subj_result_folder)

    # determine list of fe names
    subj_fe_names = []
    for fe_cls, fe_settings in zip(feature_extractor_classes, feature_extractor_settings):
        fe_name = f'{fe_cls.__name__}_{hash_configs(fe_settings)}'
        subj_fe_names.append(fe_name)

    # check if it already exists
    if not overwrite:
        json_path = os.path.join(subj_result_folder, 'fe_names.json')
        if os.path.exists(json_path):
            with open(json_path) as f:
                existing_fe_names = json.load(f)
            if existing_fe_names == subj_fe_names:
                print(f'Subject {subj_id} already processed')
                return
    
    subj_features = []
    subj_masked_df = None
    for fe_cls, fe_settings in zip(feature_extractor_classes, feature_extractor_settings):
        fe_name = f'{fe_cls.__name__}_{hash_configs(fe_settings)}'
        fe = fe_cls(fe_settings)
        features, _, subj_masked_df, _  = read_one_subject_feature(
            fe, source_eyedata, source_behav_df, subj_id)
        
        fe_folder_path = os.path.join(subj_result_folder, fe_name)
        if not os.path.exists(fe_folder_path):
            os.makedirs(fe_folder_path)

        # save the settings
        config_path = os.path.join(fe_folder_path, 'settings.json')
        with open(config_path, 'w') as fp:
            json.dump(fe_settings, fp, indent=4)

        # save the features
        feature_path = os.path.join(fe_folder_path, 'features.npy')
        np.save(feature_path, features)

        # finally, add it to the combined features
        subj_features.append(features)

        
    # create the combined features
    subj_features = np.concatenate(subj_features, axis=-1)
    # reshape: time x trials x n_feature
    subj_features = np.transpose(subj_features, axes=(1, 0, 2))
    # save the combined_features
    subj_combined_feature_path = os.path.join(subj_result_folder, 'combined')
    if not os.path.exists(subj_combined_feature_path):
        os.makedirs(subj_combined_feature_path)

    for tid in range(len(subj_features)):
        np.save(
            os.path.join(subj_combined_feature_path, f'{tid}.npy'), 
            subj_features[tid])

    # save the behavior file
    subj_behav_psth = os.path.join(subj_result_folder, 'behavior.csv')
    subj_masked_df.to_csv(subj_behav_psth)

    # save the list of settings
    with open(os.path.join(subj_result_folder, 'fe_names.json'), 'w') as fp:
        json.dump(subj_fe_names, fp)

In [28]:
REFORMATTED_XY = [create_xydata_from_df(df) for df in AllGazeCleans]

In [29]:
DEFAULT_FE_CLASSES = [
    GazeMeanExtractor,
    GazeAngleFeatureExtractor,
    GazeHeatmapFeatureExtractor,
]

DEFAULT_FE_SETTINGS = [
    DEFAULT_MEAN_POS_SETTINGS,
    DEFAULT_1D_VEC_SETTINGS,
    DEFAULT_2D_VEC_SETTINGS,
]


In [30]:
OVERWRITE = True

for batch_id, valid_subjs in enumerate(AllSubjBatches):
    for subj in valid_subjs:
        print(f'generating features for {subj}')
        generate_combined_features_for_one_subjects(
            feature_extractor_classes=DEFAULT_FE_CLASSES, 
            feature_extractor_settings=DEFAULT_FE_SETTINGS,
            source_eyedata=REFORMATTED_XY[batch_id],
            source_behav_df=AllPsyFull[batch_id], subj_id=subj,
            overwrite=OVERWRITE)

generating features for 908
generating features for 909
generating features for 910
generating features for 912
generating features for 913
generating features for 914
generating features for 915
generating features for 916
generating features for 917
generating features for 918
generating features for 920
generating features for 921
generating features for 922
generating features for 927
generating features for 929
generating features for 930
generating features for 931
generating features for 932
generating features for 933
generating features for 934
generating features for 935
generating features for 937
generating features for 938
generating features for 939
generating features for 940
generating features for 941
generating features for 942
generating features for 943
generating features for 944
generating features for 945
generating features for 946
generating features for 947
generating features for 948
generating features for 949
generating features for 950
generating features 